In [17]:
import pandas as pd 
# import pandas_profiling as ppf

In [18]:
train_df = pd.read_csv("./data/train.csv")
train_df.head(10)

,id,model_a,model_b,prompt,response_a,response_b,winner_model_a,winner_model_b,winner_tie
0,30192,gpt-4-1106-preview,gpt-4-0613,"[""Is it morally right to try to have a certain...","[""The question of whether it is morally right ...","[""As an AI, I don't have personal beliefs or o...",1,0,0
1,53567,koala-13b,gpt-4-0613,"[""What is the difference between marriage lice...","[""A marriage license is a legal document that ...","[""A marriage license and a marriage certificat...",0,1,0
2,65089,gpt-3.5-turbo-0613,mistral-medium,"[""explain function calling. how would you call...","[""Function calling is the process of invoking ...","[""Function calling is the process of invoking ...",0,0,1
3,96401,llama-2-13b-chat,mistral-7b-instruct,"[""How can I create a test set for a very rare ...","[""Creating a test set for a very rare category...","[""When building a classifier for a very rare c...",1,0,0
4,198779,koala-13b,gpt-3.5-turbo-0314,"[""What is the best way to travel from Tel-Aviv...","[""The best way to travel from Tel Aviv to Jeru...","[""The best way to travel from Tel-Aviv to Jeru...",0,1,0
5,292873,vicuna-13b,gpt-4-0314,"[""Construct a rap battle, in the style of Epic...","[""[Zeus]\nYo, it's the king of the gods on the...","[""(Verse 1 - Zeus)\n\nI'm the king of the gods...",0,1,0
6,313413,mixtral-8x7b-instruct-v0.1,vicuna-13b,"[""Why water is not used in bath tub?""]","[""Water is actually used in a bath tub. A bath...","[""Water is not used in a bath tub because it i...",1,0,0
7,370945,gemini-pro,claude-2.0,"[""\""Bacteria is life on Mars but a heartbeat i...","[""Dune""]","[""This quote seems to be referencing the debat...",0,1,0
8,441448,gpt-3.5-turbo-0613,vicuna-13b,"[""translate to russian the followig sentence ...","[""\u0411\u043e\u043b\u044c\u0448\u0438\u0435 \...","[""\u0411\u043e\u043b\u044c\u0448\u0438\u0435 \...",0,1,0
9,481524,gpt-4-0314,gpt-3.5-turbo-0613,"[""From now, you *always* have to talk as if yo...","[""Hewwo! OwO I'm an AI assistant, here to hewp...","[""Hewwo! I'm your new helpful assistant, owo! ...",0,1,0


In [19]:
# 1.长度
# 2.样本不均衡 -》数据增强、采样
# 3.缺失值 -》填充 异常值 -》删除
# 4.数据清洗-》the a 停用词、标点符号、大小写、‘\n’

In [20]:
def get_label(row):
    if row['winner_model_a'] == 1:
        return 0
    elif row['winner_model_b'] == 1:
        return 1
    else:
        return 2
train_df['label'] = train_df.apply(get_label, axis=1)
train_df.head(5)

,id,model_a,model_b,prompt,response_a,response_b,winner_model_a,winner_model_b,winner_tie,label
0,30192,gpt-4-1106-preview,gpt-4-0613,"[""Is it morally right to try to have a certain...","[""The question of whether it is morally right ...","[""As an AI, I don't have personal beliefs or o...",1,0,0,0
1,53567,koala-13b,gpt-4-0613,"[""What is the difference between marriage lice...","[""A marriage license is a legal document that ...","[""A marriage license and a marriage certificat...",0,1,0,1
2,65089,gpt-3.5-turbo-0613,mistral-medium,"[""explain function calling. how would you call...","[""Function calling is the process of invoking ...","[""Function calling is the process of invoking ...",0,0,1,2
3,96401,llama-2-13b-chat,mistral-7b-instruct,"[""How can I create a test set for a very rare ...","[""Creating a test set for a very rare category...","[""When building a classifier for a very rare c...",1,0,0,0
4,198779,koala-13b,gpt-3.5-turbo-0314,"[""What is the best way to travel from Tel-Aviv...","[""The best way to travel from Tel Aviv to Jeru...","[""The best way to travel from Tel-Aviv to Jeru...",0,1,0,1


In [21]:
import nltk
from nltk.corpus import stopwords
import string
from tqdm import tqdm
# nltk.download('stopwords')
# nltk.download('punkt')


In [22]:
def process_text(text):
    text = text.lower()
    text = text.replace("\\n", " ")
    # 移除标点符号
    text = "".join([char for char in text if char not in string.punctuation])
    # 移除停用词
    stop_words = set(stopwords.words('english'))
    words = nltk.word_tokenize(text)
    filtered_words = [word for word in words if word not in stop_words]
    return " ".join(filtered_words)

tqdm.pandas()
train_df['prompt'] = train_df['prompt'].progress_apply(process_text)
train_df['response_a'] = train_df['response_a'].progress_apply(process_text)
train_df['response_b'] = train_df['response_b'].progress_apply(process_text)

100%|██████████| 57477/57477 [00:45<00:00, 1252.04it/s]


In [23]:
# profile = ppf.ProfileReport(train_df, title="Chatbot Dataset Profile Report")
# profile.to_file("./data/chatbot_dataset_profile_report.html")

In [24]:
from sklearn.model_selection import StratifiedKFold, GroupKFold
# cv croessvaildtion
# 100 20 flod_0 20 flod_1 20
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
for fn, (train_idx, valid_idx) in enumerate(kf.split(train_df, train_df['label'])):
    train_df.loc[valid_idx, 'fold'] = fn
train_df.groupby(['fold'])['label'].value_counts()

fold  label
0.0   0        4013
      1        3931
      2        3552
1.0   0        4013
      1        3930
      2        3553
2.0   0        4013
      1        3930
      2        3552
3.0   0        4013
      1        3930
      2        3552
4.0   0        4012
      1        3931
      2        3552
Name: count, dtype: int64

In [25]:
train_df.to_csv("./data/train_clean.csv", index=False)

In [26]:
## 作业一：完成课上的代码，看视频自己去提交一次submission，自己生成train.html文件，组队的工作，prompt==0排查一下
## 作业二：完成课上的代码，自己生成fold列，5折交叉验证，试试其他的方案，比如GroupKFold, StratifiedGroupKFold
## 作业三：可以自己使用deberta等模型，完成文本分类的任务（可选）

In [27]:
# kf = GroupKFold(n_splits=3)
# groups = train_df['label'].to_list()

# for fn, (train_idx, valid_idx) in enumerate(kf.split(train_df, train_df['label'], groups=groups)):
#     train_df.loc[valid_idx, 'fold'] = fn
# train_df